In [49]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Loading results dataframes for use in linear regression modeling
run_data = pd.read_csv("data/stream-data/raw-stream-data/russian.csv", thousands=',')
days_below_32 = pd.read_csv("data/weather-data/KenaiAirport_Days-Below-32F.csv")
summer_mean_temps = pd.read_csv("data/weather-data/KenaiAirport_Summer-Mean-Temps-Yearly.csv")
winter_mean_temps = pd.read_csv("data/weather-data/KenaiAirport_Winter-Mean-Temps-Yearly.csv")
input_data = pd.merge(left=days_below_32, right=summer_mean_temps, left_on='year', right_on='year')
input_data = pd.merge(left=input_data, right=winter_mean_temps, left_on='year', right_on='year')

Now that we have our data loaded and the proper modules imported, let's process the Russian
River run datafile to parse out our target output variables (run counts) as well as gather a 
few more potential input variables (effort counts or "angler days").

In [50]:
drop_columns = ['ER  Escapement', 'LR Escapement', 'ER', 'LR']
run_data.drop(columns=drop_columns, inplace=True)
run_data.rename(columns={'Year' : 'year'}, inplace=True)
# merge with the input_data frame
model_data = pd.merge(left=input_data, right=run_data, left_on='year', right_on='year')
# dropping the year with no Effort value
model_data.drop(index=0, inplace=True)
model_data['Effort'] = model_data['Effort'].str.replace(',', '').astype(int)



year                  int64
days_below_32         int64
mean_summer_temp    float64
mean_winter_temp    float64
Effort                int32
ER Sport              int64
LR Sport              int64
ER Local              int64
LR Local              int64
Total Run             int64
dtype: object